In [3]:
import sys
import os
# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [4]:
import numpy as np
from env.init import initialize_environment
from models.recurrent_ppo import RecurrentPPOModel
from env.states_generator import StatesGenerator
import copy

In [5]:
env, config = initialize_environment()
model = RecurrentPPOModel.load("../mlruns/350479807605635968/616b2a0286cc492a8e5df0ffd3af589b/artifacts/models/best_model.zip", env, config)
original_states = env.generate_states()

/home/ediss5/anaconda3/envs/cades/lib/python3.9/site-packages/stable_baselines3/common/env_checker.py:190: UserWarning: Your observation communications has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [6]:
def generate_rand_comm_tree(tasks, critical_mask, tasks_lens):
    # create states generator
    states_generator = StatesGenerator(config)
    # generate random communication tree
    comm = states_generator.generate_chained_communications(
        tasks,
        critical_mask,
        tasks_lens,
    )
    return comm

In [11]:
num_evaluations = 10
for i in range(num_evaluations):
    print(f"Evaluation {i}:")
    new_states = copy.deepcopy(original_states)
    comms, comms_len = generate_rand_comm_tree(original_states["tasks"], original_states["critical_mask"], original_states["tasks_lens"])
    new_states["communications"] = comms
    new_states["communications_lens"] = comms_len
    print(new_states["communications"][0])
    result = model.evaluate(new_states)
    print(result)

Evaluation 0:
[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 1 0 0 1 0 0 0 0 0]]
{'obs': {'tasks': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'critical_mask': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), 'nodes': array([6.66666667e-01, 6.66666667e-01, 1.11022302e-16, 3.33333333e-01,
       6.66666667e-01, 3.33333333e-01, 6.66666667e-01, 6.66666667e-01]), 'communications': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 